In [1]:
from flask import Flask, request
import sys
sys.path.append('..')
from dataset import DataLoader, DatasetBuilder
#from models import *
from utils.input import *
import sys 
sys.path.append('..')
from models import *
from model import *
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid", font_scale=1.5, palette='magma')
import matplotlib
matplotlib.use('agg')

/Users/zattacole/anaconda3/envs/zatta/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
def input_pipeline(path:str='../DATA/speech_commands_v0.02',
                   method_spectrum:str='log_mel',
                   test_ratio:float=0.15,
                   val_ratio:float=0.05,
                   batch_size:int=64,
                   shuffle_buffer_size:int=1000,
                   shuffle:bool=True,
                   seed:int=42,
                   verbose:int=1):
    """
    Get the data.
    
    Parameters
    ----------
    path : str
        Path to the data.
    method_spectrum : str
        Method to compute the spectrum.
    test_ratio : float
        Ratio of the data to be used as test set.
    val_ratio : float
        Ratio of the data to be used as validation set.
    batch_size : int
        Batch size.
    shuffle_buffer_size : int
        Shuffle buffer size.
    shuffle : bool
        Whether to shuffle the data.
    seed : int
        Seed for the random number generator.
    verbose : int
        Verbosity level.
        
    Returns
    -------
    train : tf.data.Dataset
        Training dataset.
    test : tf.data.Dataset
        Test dataset.
    val : tf.data.Dataset
        Validation dataset.
    commands : list
        List of commands.
    """

    # Get the files.
    data = DataLoader(
        path=path
    )
    
    commands = data.get_commands()
    filenames = data.get_filenames()
    train_files, test_files, val_files = data.split_data(
        filenames=filenames,
        test_ratio=test_ratio,
        val_ratio=val_ratio,
        shuffle=shuffle,
        seed=seed,
        verbose=verbose
    )

    ds = DatasetBuilder(
        commands=commands,
        train_filenames=train_files,
        test_filenames=test_files,
        val_filenames=val_files,
        batch_size=batch_size,
        buffer_size=shuffle_buffer_size,
        method=method_spectrum
    )
    
    train, test, val = ds.preprocess_dataset_spectrogram()
    return train, test, val, commands

def evaluation_pipeline(
    model_name:str,
    model:tf.keras.Model,
    test_ds:tf.data.Dataset,
    commands:list,
    verbose:int=1,
):
    """
    Evaluate the model.
    
    Parameters
    ----------
    model : tf.keras.Model
        Trained model.
    test_ds : tf.data.Dataset
        Test dataset.
    commands : list
        List of commands.
    verbose : int
        Verbosity level.
    """

    
    # Evaluate the model.   
    mtest = model.evaluate(
            set='test',
            method='confusion_matrix',
            model_name=model_name,
        )
       
    # Save the metrics.
    return mtest

import models
def training_pipeline(
    name_model:str,
    train_ds:tf.data.Dataset,
    test_ds:tf.data.Dataset,
    val_ds:tf.data.Dataset,
    commands:list,
    loss:str,
    optimizer:str,
    metrics:str,
    epochs:int=300,
    use_tensorboard:bool=True,
    save_checkpoint:bool=True,
    verbose:int=1,
):
    """
    Get the model, compile it, train it and evaluate it.
    """

    # Get the model.
    model = getattr(models, name_model)(
        train_ds=train_ds,
        test_ds=test_ds,
        val_ds=val_ds,
        commands=commands
    )

    if verbose:
        print('Model: {}'.format(name_model))

    model.define_model()
    model.load_weights('logmel/models/CNNOneTStride8.h5')
   
    
    return model

In [13]:
def main(
    path='DATA/speech_commands_v0.02',
    method_spectrum='STFT',
    test_ratio=0.15,
    val_ratio=0.05,
    batch_size=128,
    shuffle_buffer_size=1000,
    name_model='CNNOneTStride8',
    loss='sparse_categorical_crossentropy',
    lr=0.01,
    metrics='accuracy',
    epochs=300,
    shuffle=True,
    use_tensorboard:bool=True,
    save_checkpoint:bool=True,
    verbose=1,
    seed=42,
):
    """
    Main function. Get the data, train the model and evaluate it.
    
    Parameters
    ----------
    path : str
        Path to the data.
    method_spectrum : str
        Method to compute the spectrum.
    test_ratio : float
        Ratio of the data to be used as test set.
    val_ratio : float
        Ratio of the data to be used as validation set.
    batch_size : int
        Batch size.
    shuffle_buffer_size : int
        Shuffle buffer size.
    name_model : str
        Name of the model.
    loss : str
        Loss function.
    optimizer : str
        Optimizer.
    metrics : str
        Metrics.
    epochs : int
        Number of epochs.
    seed : int
        Seed for the random number generator.
    verbose : int
        Verbosity level.
    """

    # print args
    print('path: {}'.format(path))
    print('method_spectrum: {}'.format(method_spectrum))
    print('test_ratio: {}'.format(test_ratio))
    print('val_ratio: {}'.format(val_ratio))
    print('batch_size: {}'.format(batch_size))
    print('shuffle_buffer_size: {}'.format(shuffle_buffer_size))
    print('name_model: {}'.format(name_model))
    print('loss: {}'.format(loss))
    print('lr: {}'.format(lr))
    print('metrics: {}'.format(metrics))
    print('epochs: {}'.format(epochs))
    print('shuffle: {}'.format(shuffle))
    print('use_tensorboard: {}'.format(use_tensorboard))
    print('save_checkpoint: {}'.format(save_checkpoint))


    # optimizer = tf.keras.optimizers.Adam(learning_rate=float(lr), weight_decay=1e-5)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=float(lr), decay=1e-5)

    train, test, val, commands = input_pipeline(
        path=path,
        method_spectrum=method_spectrum,
        test_ratio=test_ratio,
        val_ratio=val_ratio,
        batch_size=batch_size,
        shuffle_buffer_size=shuffle_buffer_size,
        shuffle=shuffle,
        seed=seed,
        verbose=verbose
    )
    # img size
    img_size = train.element_spec
    print(img_size)

    
    model = training_pipeline(
        name_model=name_model,
        train_ds=train,
        test_ds=test,
        val_ds=val,
        loss=loss,
        optimizer=optimizer,
        metrics=metrics,
        epochs=100,
        use_tensorboard=use_tensorboard,
        save_checkpoint=save_checkpoint,
        verbose=1,
        commands=commands,
    )

    
    mat = evaluation_pipeline(
        model_name=name_model,
        model=model,
        test_ds=test,
        commands=commands,
        verbose=1
    )
    return mat, commands



In [14]:
m, command = main()

path: DATA/speech_commands_v0.02
method_spectrum: STFT
test_ratio: 0.15
val_ratio: 0.05
batch_size: 128
shuffle_buffer_size: 1000
name_model: CNNOneTStride8
loss: sparse_categorical_crossentropy
lr: 0.01
metrics: accuracy
epochs: 300
shuffle: True
use_tensorboard: True
save_checkpoint: True
Dataset path:  DATA/speech_commands_v0.02
(TensorSpec(shape=(None, 257, None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))


2023-06-11 17:13:15.389387: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Model: CNNOneTStride8
2/2 [==============================] - 0s 2ms/step


In [16]:
# normalize rows 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(font_scale=1.5, palette="Blues")
m, command = m
mat_norm = m/np.sum(m, axis=0)

In [20]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(mat_norm, ax=ax, cmap='Blues')

ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title('Confusion Matrix')
fig.savefig('confusion_matrix.pdf', dpi=300, bbox_inches='tight')
